##imports

In [1]:
import pandas as pd
import altair as alt
import kagglehub

## kaggle datset for lattitude

In [2]:
# Download latest version
path = kagglehub.dataset_download("hserdaraltan/countries-by-continent")

print("Path to dataset files:", path)
continent_df= pd.read_csv(
    "/kaggle/input/countries-by-continent/Countries by continents.csv"
)
continent_df.head()

Using Colab cache for faster access to the 'countries-by-continent' dataset.
Path to dataset files: /kaggle/input/countries-by-continent


,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina


In [3]:
import os
repo_url = "https://github.com/est-her393/data-visual-final.git"
repo_dir = "data-visual-final"

if os.path.exists(repo_dir) and os.listdir(repo_dir):
    print(f"Directory '{repo_dir}' already exists. Skipping clone.")
else:
    print(f"Cloning '{repo_url}' into '{repo_dir}'...")
    !git clone {repo_url}

print("\nFiles in repo:")
print(os.listdir(repo_dir))

Cloning 'https://github.com/est-her393/data-visual-final.git' into 'data-visual-final'...
Cloning into 'data-visual-final'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 64 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 7.47 MiB | 5.32 MiB/s, done.
Resolving deltas: 100% (28/28), done.

Files in repo:
['owid-co2-data.csv', 'annual-co2-emissions-per-country.csv', 'gain.csv', 'vulnerability.csv', 'co2-fossil-plus-land-use.csv', 'annual-co-emissions-by-region.csv', 'finalviz_jade_v1.ipynb', '.git', 'GPD.csv', 'GINI.csv', 'DVP_v3.ipynb', 'readiness.csv']


In [4]:
# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/countries-of-the-world-2023")

print("Path to dataset files:", path)


alt.data_transformers.disable_max_rows()

kaggle_df = pd.read_csv(
    "/kaggle/input/countries-of-the-world-2023/world-data-2023.csv"
)

kaggle_df.head()


Using Colab cache for faster access to the 'countries-of-the-world-2023' dataset.
Path to dataset files: /kaggle/input/countries-of-the-world-2023


,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
0,Afghanistan,60,AF,58.10%,"652,230","323,000",32.49,93.0,Kabul,"8,672",...,78.40%,0.28,"38,041,754",48.90%,9.30%,71.40%,11.12%,"9,797,273",33.939110,67.709953
1,Albania,105,AL,43.10%,"28,748","9,000",11.78,355.0,Tirana,"4,536",...,56.90%,1.20,"2,854,191",55.70%,18.60%,36.60%,12.33%,"1,747,593",41.153332,20.168331
2,Algeria,18,DZ,17.40%,"2,381,741","317,000",24.28,213.0,Algiers,"150,006",...,28.10%,1.72,"43,053,054",41.20%,37.20%,66.10%,11.70%,"31,510,100",28.033886,1.659626
3,Andorra,164,AD,40.00%,468,NaN,7.20,376.0,Andorra la Vella,469,...,36.40%,3.33,"77,142",NaN,NaN,NaN,NaN,"67,873",42.506285,1.521801
4,Angola,26,AO,47.50%,"1,246,700","117,000",40.73,244.0,Luanda,"34,693",...,33.40%,0.21,"31,825,295",77.50%,9.20%,49.10%,6.89%,"21,061,025",-11.202692,17.873887


#visualization curration

In [5]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

# ======================================
# 1 loading daa
# ======================================
gini_df = pd.read_csv(f"{repo_dir}/GINI.csv")
co2_df  = pd.read_csv(f"{repo_dir}/owid-co2-data.csv")
coords  = pd.read_csv("/kaggle/input/countries-by-continent/Countries by continents.csv")
world_df = pd.read_csv("/kaggle/input/countries-of-the-world-2023/world-data-2023.csv")

# EXPECTED COLUMNS (testing ):
# gini_df:  'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960'..'2024'
# co2_df:   'country', 'year', 'co2', ...
# coords:   'Country', 'Continent', ...
# world_df: 'Country', 'Latitude', 'Longitude', ...

# ======================================
# 2. Normalize country names
# ======================================
def norm(x):
    return str(x).strip().lower()

gini_df = gini_df.copy()
co2_df  = co2_df.copy()
coords  = coords.copy()
world_df = world_df.copy()

gini_df["country_norm"]  = gini_df["Country Name"].apply(norm)
co2_df["country_norm"]   = co2_df["country"].apply(norm)
coords["country_norm"]   = coords["Country"].apply(norm)
world_df["country_norm"] = world_df["Country"].apply(norm)

# ======================================
# 3. Get latest GINI per country (from year columns)
# ======================================
year_cols = [c for c in gini_df.columns if c.isdigit()]

gini_long = gini_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code", "country_norm"],
    value_vars=year_cols,
    var_name="year",
    value_name="GINI"
)


gini_latest = (
    gini_long.sort_values("year")
             .dropna(subset=["GINI"])
             .groupby("country_norm")
             .tail(1)[["country_norm", "Country Name", "GINI", "year"]]
)

# ======================================
# 4. Get latest CO2 per country
# ======================================
co2_latest = (
    co2_df.sort_values("year")
          .dropna(subset=["co2"])
          .groupby("country_norm")
          .tail(1)[["country_norm", "country", "year", "co2"]]
          .rename(columns={"co2": "CO2"})
)

# ======================================
# 5. Prep continent + coordinates
# ======================================
coords_slim = coords[["country_norm", "Continent"]]

world_coords = world_df[["country_norm", "Country", "Latitude", "Longitude"]].rename(
    columns={"Country": "Country_latlon"}
)

# ======================================
# 6. Merge into ONE master dataframe
# ======================================
df = (
    gini_latest.rename(columns={"Country Name": "Country"})
      .merge(co2_latest[["country_norm", "CO2"]], on="country_norm", how="left")
      .merge(world_coords, on="country_norm", how="left")
      .merge(coords_slim, on="country_norm", how="left")
)

# Fix coords so they match the projection
df["Latitude"]  = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
# If longitudes are in [0, 360), wrap them to [-180, 180]
df.loc[df["Longitude"] > 180, "Longitude"] = df.loc[df["Longitude"] > 180, "Longitude"] - 360

# ======================================
# 6b. Clean final country names + harmonize for topojson
# ======================================
df["Country_final"] = df["Country"].astype(str).str.strip()

# Base: start with Country_final
df["country_map"] = df["Country_final"]

# Apply specific overrides to match Natural Earth topojson
df.loc[df["Country_final"] == "United States", "country_map"] = "United States of America"
df.loc[df["Country_final"] == "Democratic Republic of Congo", "country_map"] = "Democratic Republic of the Congo"
df.loc[df["Country_final"] == "Congo", "country_map"] = "Republic of the Congo"
df.loc[df["Country_final"] == "Czechia", "country_map"] = "Czech Republic"
df.loc[df["Country_final"] == "Eswatini", "country_map"] = "Swaziland"
df.loc[df["Country_final"] == "North Macedonia", "country_map"] = "Macedonia"
df.loc[df["Country_final"] == "South Korea", "country_map"] = "Korea, South"
df.loc[df["Country_final"] == "North Korea", "country_map"] = "Korea, North"
df.loc[df["Country_final"] == "Cape Verde", "country_map"] = "Cabo Verde"
df.loc[df["Country_final"] == "Ivory Coast", "country_map"] = "Côte d'Ivoire"
df.loc[df["Country_final"] == "Micronesia (country)", "country_map"] = "Micronesia"

# Metric for labels: CO2 per inequality
df["CO2_per_GINI"] = df["CO2"] / df["GINI"]

# Explicit domain for GINI color scale
gini_min = float(df["GINI"].min())
gini_max = float(df["GINI"].max())

# ======================================
# 7. Choropleth: GINI (yellow→red) + graticule
# ======================================
world_topo = alt.topo_feature(
    "https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json",
    "countries"
)

# Background grid
graticule = (
    alt.Chart(alt.graticule())
      .mark_geoshape(
          fill=None,
          stroke="lightgray",
          strokeWidth=0.4
      )
)

# Main choropleth
choro = (
    alt.Chart(world_topo)
      .mark_geoshape(stroke="white", strokeWidth=0.4)
      .transform_lookup(
          lookup="properties.name",
          from_=alt.LookupData(
              df,
              key="country_map",
              fields=["Country_final", "Continent", "GINI", "CO2", "CO2_per_GINI"]
          )
      )
      .encode(
          color=alt.condition(
              "datum.GINI != null",
              alt.Color(
                  "GINI:Q",
                  title="Income inequality (GINI)",
                  scale=alt.Scale(
                      scheme="yelloworangered",
                      domain=[gini_min, gini_max],
                      nice=False
                  ),
                  legend=alt.Legend(
                      orient="bottom",
                      title="Income inequality (GINI)",
                      tickCount=5
                  )
              ),
              alt.value("#4d4d4d")
          ),
          tooltip=[
              alt.Tooltip("Country_final:N",  title="Country"),
              alt.Tooltip("Continent:N"),
              alt.Tooltip("GINI:Q",          title="GINI",      format=".2f"),
              alt.Tooltip("CO2:Q",           title="CO₂",       format=".2f"),
              alt.Tooltip("CO2_per_GINI:Q",  title="CO₂ / GINI",format=".2f")
          ]
      )
)

# ======================================
# 8. Bubbles + labels for top 12 CO2_per_GINI
# ======================================
bubble_df = df.dropna(subset=["Latitude", "Longitude", "CO2_per_GINI"]).copy()
bubble_df = bubble_df.sort_values("CO2_per_GINI", ascending=False).head(12)

bubbles = (
    alt.Chart(bubble_df)
      .mark_circle(opacity=0.85)
      .encode(
          longitude="Longitude:Q",
          latitude="Latitude:Q",
          size=alt.Size(
              "CO2_per_GINI:Q",
              title="CO₂ per GINI index",
              scale=alt.Scale(range=[300, 2000]),
              legend=alt.Legend(
                  title="CO₂ per GINI index",
                  orient="right"
              )
          ),
          color=alt.value("#1a9850"),
          tooltip=[
              alt.Tooltip("Country_final:N", title="Country"),
              alt.Tooltip("Continent:N"),
              alt.Tooltip("GINI:Q",         title="GINI",        format=".2f"),
              alt.Tooltip("CO2:Q",          title="CO₂",         format=".2f"),
              alt.Tooltip("CO2_per_GINI:Q", title="CO₂ per GINI",format=".2f")
          ]
      )
)
labels = (
    alt.Chart(bubble_df)
      .mark_text(
          align="center",
          baseline="middle",
          fontSize=9,
          fontWeight="bold",
          opacity=1
      )
      .encode(
          longitude="Longitude:Q",
          latitude="Latitude:Q",
          text=alt.Text("CO2_per_GINI:Q", format=".1f")
      )
)


# ======================================
# 9. Continent labels
# ======================================
continent_centers = pd.DataFrame({
    "Continent": ["North America", "South America", "Europe", "Africa", "Asia"],
    "Latitude":  [ 50,             -20,             55,       0,        30,     ],
    "Longitude": [-100,            -60,             15,       20,       90,    ]
})

continent_labels = (
    alt.Chart(continent_centers)
      .mark_text(
          fontSize=12,
          fontWeight="lighter",
          opacity=0.35
      )
      .encode(
          longitude="Longitude:Q",
          latitude="Latitude:Q",
          text="Continent:N"
      )
)

# ======================================
# 10. Final full-world map
# ======================================
final_map = (
    graticule + choro + bubbles + labels + continent_labels
).project(
    type="equalEarth"
).properties(
    width=1000,
    height=520,
    title="Top 10 Global Inequality vs CO₂ Emissions (Full-World View)"
).configure_view(
    stroke=None
).configure_legend(
    labelFontSize=11,
    titleFontSize=12
)

final_map


alt.LayerChart(...)

In [6]:
import altair as alt

# [Assuming 'df' is loaded]

# === METHOD 4: GEO-BUBBLE "FULL WORLD" MAP ===

# 1. The Background (The "World" Context)
# We draw the land in light gray so it doesn't compete with the data
world_topo = alt.topo_feature(
    "https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json",
    "countries"
)

background = alt.Chart(world_topo).mark_geoshape(
    fill='#f0f0f0',      # Very light gray fill
    stroke='white',      # White borders
    strokeWidth=0.5
).project(
    type='equalEarth'    # "Full World" projection that respects area sizes
)

# 2. The Data Layer (Bubbles)
# We plot circles at the country's Latitude/Longitude
bubbles = alt.Chart(df).mark_circle(
    opacity=0.8,
    stroke='white',
    strokeWidth=0.5
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',

    # SIZE = CO2 Emissions
    size=alt.Size(
        'CO2:Q',
        scale=alt.Scale(range=[10, 1000]),
        title='CO2 Emissions',
        legend=alt.Legend(orient='bottom-right') # Move legend out of the way
    ),

    # COLOR = Inequality (GINI)
    color=alt.Color(
        'GINI:Q',
        scale=alt.Scale(scheme='magma'), # 'Magma' is great for dark-to-light intensity
        title='Inequality (GINI)',
        legend=alt.Legend(orient='bottom-left')
    ),

    tooltip=[
        alt.Tooltip('Country_final', title='Country'),
        alt.Tooltip('GINI', format='.1f'),
        alt.Tooltip('CO2', format='.1f'),
        alt.Tooltip('Continent')
    ]
)

# 3. Add Interaction: Zoom & Pan
# This makes the "Full World View" explorable
zoom = alt.selection_interval(bind='scales')

# 4. Combine
final_map = (background + bubbles).add_params(
    zoom
).properties(
    width=800,
    height=500,
    title="Global Inequality vs CO2 (Geo-Bubble View)"
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start'
)

final_map

alt.LayerChart(...)

In [7]:
import pandas as pd
import altair as alt
import numpy as np

alt.data_transformers.disable_max_rows()

# ======================================
# 1. Load Data
# ======================================
# Replace these paths with your actual file paths
try:
    gini_df = pd.read_csv(f"{repo_dir}GINI.csv")
    co2_df  = pd.read_csv(f"{repo_dir}owid-co2-data.csv")
    world_df = pd.read_csv("/kaggle/input/countries-of-the-world-2023/world-data-2023.csv")
except FileNotFoundError:
    print("ERROR: Please update file paths in Section 1 before running.")

# Helper for normalization
def norm(x): return str(x).strip().lower()

# ======================================
# 2. Process GINI (Long Format & Time Series Prep)
# ======================================
gini_df["country_norm"] = gini_df["Country Name"].apply(norm)
year_cols = [c for c in gini_df.columns if c.isdigit()]

# Melt to long format: Country | Year | GINI
gini_long = gini_df.melt(
    id_vars=["country_norm", "Country Name"],
    value_vars=year_cols,
    var_name="year",
    value_name="GINI"
)
gini_long['year'] = gini_long['year'].astype(int)
gini_long = gini_long.dropna(subset=['GINI'])

# ======================================
# 3. Process CO2 (Long Format)
# ======================================
co2_df["country_norm"] = co2_df["country"].apply(norm)
co2_long = co2_df[['country_norm', 'year', 'co2']].rename(columns={'co2': 'CO2'})
co2_long = co2_long.dropna(subset=['CO2'])

# ======================================
# 4. The Crucial Step: Merging & Filling Time Gaps
# ======================================
# Define the timeframe we want to animate (e.g., 1990 to 2020 holds good overlap)
min_year, max_year = 1990, 2020
years_range = pd.DataFrame({'year': range(min_year, max_year + 1)})

# Get unique countries
unique_countries = pd.DataFrame({'country_norm': gini_long['country_norm'].unique()})

# Create a skeleton grid: Every country for every year in the range
# This ensures the animation runs smoothly even if data is missing for a year
skeleton = unique_countries.merge(years_range, how='cross')

# Merge GINI onto skeleton and forward-fill missing years
df_time = skeleton.merge(gini_long[['country_norm', 'year', 'GINI', 'Country Name']],
                         on=['country_norm', 'year'], how='left')
df_time = df_time.sort_values(['country_norm', 'year'])
# Forward fill: assumes GINI stays same until a new measurement appears
df_time['GINI'] = df_time.groupby('country_norm')['GINI'].ffill()

# Merge CO2 onto the time series Result
df_time = df_time.merge(co2_long, on=['country_norm', 'year'], how='left')

# Filter out rows that still have N/A after filling (e.g., years before first GINI measurement)
df_time = df_time.dropna(subset=['GINI', 'CO2', 'Country Name'])

# ======================================
# 5. Add Coordinates
# ======================================
world_df["country_norm"] = world_df["Country"].apply(norm)
world_coords = world_df[["country_norm", "Latitude", "Longitude"]].copy()
world_coords["Latitude"] = pd.to_numeric(world_coords["Latitude"], errors="coerce")
world_coords["Longitude"] = pd.to_numeric(world_coords["Longitude"], errors="coerce")
# Wrap longitudes
world_coords.loc[world_coords["Longitude"] > 180, "Longitude"] = world_coords.loc[world_coords["Longitude"] > 180, "Longitude"] - 360

# Final merge
df_final = df_time.merge(world_coords, on="country_norm", how="inner")



# ======================================
# 6. VISUALIZATION: Animated Map with Continent Labels
# ======================================

# A. Create the Slider Widget mechanism
slider = alt.binding_range(min=min_year, max=max_year, step=1, name='Select Year: ')
select_year = alt.selection_point(fields=['year'], bind=slider, value=2000)

# B. The Background Map (Static)
world_topo = alt.topo_feature("https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json", "countries")
background = alt.Chart(world_topo).mark_geoshape(
    fill='#f0f0f0', stroke='white', strokeWidth=0.5
).project(type='equalEarth')

# C. Continent Labels (Static Layer)
# We define manual positions for clarity so they don't overlap with countries
continent_data = pd.DataFrame({
    'Continent': ['North America', 'South America', 'Europe', 'Africa', 'Asia', 'Oceania'],
    'Latitude':  [50,              -15,             54,       5,        40,     -25],
    'Longitude': [-100,            -60,             15,       20,       90,     140]
})

continent_labels = alt.Chart(continent_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=14,
    fontWeight='bold',
    opacity=0.3,   # Low opacity so they look like a watermark
    color='gray'
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    text='Continent:N'
)

# D. The Bubbles (Dynamic - Filtered by Slider)
bubbles = alt.Chart(df_final).mark_circle(
    stroke='white',
    strokeWidth=0.5,
    opacity=0.8
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    # SIZE = CO2
    size=alt.Size('CO2:Q', scale=alt.Scale(domain=[0, df_final['CO2'].max()], range=[10, 1500]), legend=None),
    # COLOR = GINI
    color=alt.Color('GINI:Q', scale=alt.Scale(scheme='magma', domain=[25, 65]), title='Inequality (GINI)'),
    tooltip=[
        alt.Tooltip('Country Name', title='Country'),
        alt.Tooltip('year', title='Year'),
        alt.Tooltip('GINI', format='.1f'),
        alt.Tooltip('CO2', format=',.0f', title='CO2 Emissions (Mt)')
    ]
).add_params(
    select_year
).transform_filter(
    select_year
)

# E. Combine (Background + Labels + Bubbles)
final_animated_map = (background + continent_labels + bubbles).properties(
    width=800,
    height=500,
    title="Evolution of Inequality vs CO2 Emissions (1990-2020)"
).configure_view(
    stroke=None
).configure_title(
    fontSize=16
)

final_animated_map

ERROR: Please update file paths in Section 1 before running.


alt.LayerChart(...)

In [8]:
# ... [Keep Sections 1-5 exactly the same] ...

# ======================================
# 6. VISUALIZATION: Enhanced Animated Map
# ======================================

# A. Create the Slider Widget mechanism
slider = alt.binding_range(min=min_year, max=max_year, step=1, name='Select Year: ')
select_year = alt.selection_point(fields=['year'], bind=slider, value=2000)

# Define projection once to reuse across layers
proj_type = 'equalEarth'

# B.1 The Ocean Base (New Layer)
# Draws a sphere behind everything to represent water
sphere = alt.Chart(alt.sphere()).mark_geoshape(
    fill='#e3f2fd' # Very pale blue
).project(type=proj_type)

# B.2 Graticules/Gridlines (New Layer)
# Adds faint lat/lon lines for geographic context
graticule = alt.Chart(alt.graticule()).mark_geoshape(
    stroke='#ffffff', # White lines blend subtly with the pale ocean
    strokeWidth=0.3
).project(type=proj_type)

# B.3 The Land Background (Static)
world_topo = alt.topo_feature("https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json", "countries")
background = alt.Chart(world_topo).mark_geoshape(
    fill='#e0e0e0', # Slightly darker gray land for contrast against ocean
    stroke='white',
    strokeWidth=0.5
).project(type=proj_type)

# C. Continent Labels (Static Layer)
continent_data = pd.DataFrame({
    'Continent': ['North America', 'South America', 'Europe', 'Africa', 'Asia', 'Oceania'],
    'Latitude':  [50,              -15,             54,       5,        40,     -25],
    'Longitude': [-100,            -60,             15,       20,       90,     140]
})

continent_labels = alt.Chart(continent_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=14,
    fontWeight='bold',
    opacity=0.4,
    color='#616161' # Darker gray text
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    text='Continent:N'
)

# D. The Bubbles (Dynamic - Filtered by Slider)
bubbles = alt.Chart(df_final).mark_circle(
    stroke='white',
    strokeWidth=0.75, # Slightly thicker stroke for better definition
    opacity=0.85
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    # SIZE = CO2 (Added Legend back for visual context)
    size=alt.Size(
        'CO2:Q',
        scale=alt.Scale(domain=[0, df_final['CO2'].max()], range=[10, 1500]),
        legend=alt.Legend(title='CO2 Emissions (Mt)', orient='bottom-right', labelColor='gray', titleColor='gray')
    ),
    # COLOR = GINI
    color=alt.Color(
        'GINI:Q',
        scale=alt.Scale(scheme='magma', domain=[25, 65]),
        title='Inequality (GINI)'
    ),
    tooltip=[
        alt.Tooltip('Country Name', title='Country'),
        alt.Tooltip('year', title='Year'),
        alt.Tooltip('GINI', format='.1f'),
        alt.Tooltip('CO2', format=',.0f', title='CO2 Emissions (Mt)')
    ]
).add_params(
    select_year
).transform_filter(
    select_year
)

# E. Combine Layers (Order matters!)
# Sphere bottom -> Graticule -> Land -> Labels -> Bubbles top
final_animated_map = alt.layer(
    sphere,
    graticule,
    background,
    continent_labels,
    bubbles
).properties(
    width=800,
    height=500,
    title="Evolution of Inequality vs CO2 Emissions (1990-2020)"
).configure_view(
    stroke=None # Removes the outer box border
).configure_title(
    fontSize=16,
    anchor='start'
).configure_legend(
    gradientThickness=15,
    labelFontSize=11,
    titleFontSize=12
)

final_animated_map

alt.LayerChart(...)

In [9]:
# ... [Keep Sections 1-5 exactly the same as before] ...

# ======================================
# 6. VISUALIZATION: Final "GLOBE" Style Map (Fixed Legends)
# ======================================

# A. Create the Slider Widget mechanism
slider = alt.binding_range(min=min_year, max=max_year, step=1, name='Select Year: ')
select_year = alt.selection_point(fields=['year'], bind=slider, value=2000)

proj_type = 'equalEarth'

# B.1 OCEAN
sphere = alt.Chart(alt.sphere()).mark_geoshape(
    fill='#CDE4F7'
).project(type=proj_type)

# B.2 GRID
graticule = alt.Chart(alt.graticule()).mark_geoshape(
    stroke='#ffffff',
    strokeWidth=0.8,
    opacity=0.4
).project(type=proj_type)

# B.3 LAND
world_topo = alt.topo_feature("https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json", "countries")
background = alt.Chart(world_topo).mark_geoshape(
    fill='#F1E6D0',
    stroke='white',
    strokeWidth=0.5
).project(type=proj_type)

# C. Continent Labels
continent_data = pd.DataFrame({
    'Continent': ['North America', 'South America', 'Europe', 'Africa', 'Asia', 'Oceania'],
    'Latitude':  [50,              -15,             54,       5,        40,     -25],
    'Longitude': [-100,            -60,             15,       20,       90,     140]
})

continent_labels = alt.Chart(continent_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=13,
    fontWeight='bold',
    opacity=0.5,
    color='#5D4037'
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    text='Continent:N'
)

# D. Bubbles (With Fixed Legend)
bubbles = alt.Chart(df_final).mark_circle(
    stroke='white',       # White border on map bubbles
    strokeWidth=0.5,
    opacity=0.85
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',

    # SIZE LEGEND (Fixed)
    size=alt.Size(
        'CO2:Q',
        scale=alt.Scale(domain=[0, df_final['CO2'].max()], range=[10, 1500]),
        legend=alt.Legend(
            title='CO2 Emissions (Mt)',
            orient='right',           # Move to the right side
            symbolFillColor='gray',   # Neutral color for size reference
            symbolStrokeColor='white',# Match the map bubble style
            symbolStrokeWidth=0.5
        )
    ),

    # COLOR LEGEND
    color=alt.Color(
        'GINI:Q',
        scale=alt.Scale(scheme='magma', domain=[25, 65]),
        title='Inequality (GINI)',
        legend=alt.Legend(orient='right') # Stack with the size legend
    ),

    tooltip=[
        alt.Tooltip('Country Name', title='Country'),
        alt.Tooltip('year', title='Year'),
        alt.Tooltip('GINI', format='.1f'),
        alt.Tooltip('CO2', format=',.0f', title='CO2 Emissions (Mt)')
    ]
).add_params(
    select_year
).transform_filter(
    select_year
)

# E. Combine Layers
final_globe_map = alt.layer(
    sphere,
    graticule,
    background,
    continent_labels,
    bubbles
).properties(
    width=800,
    height=500,
    title="Global Inequality vs CO2 Emissions (Time-Lapse)"
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start'
).configure_legend(
    titleFontSize=12,
    labelFontSize=11,
    padding=10
)

final_globe_map

alt.LayerChart(...)

In [10]:
# ... [Keep Sections 1-5 exactly the same] ...

# ======================================
# 6. VISUALIZATION: "Physical Atlas" Style (Realistic Colors)
# ======================================

# A. Create the Slider Widget mechanism
slider = alt.binding_range(min=min_year, max=max_year, step=1, name='Select Year: ')
select_year = alt.selection_point(fields=['year'], bind=slider, value=2000)

proj_type = 'equalEarth'

# B.1 OCEAN (Realistic "Steel Blue")
sphere = alt.Chart(alt.sphere()).mark_geoshape(
    fill='#b8d5e5'
).project(type=proj_type)

# B.2 GRID (Subtle White Lines)
graticule = alt.Chart(alt.graticule()).mark_geoshape(
    stroke='#ffffff',
    strokeWidth=0.8,
    opacity=0.3  # Reduced opacity to blend better
).project(type=proj_type)

# B.3 LAND (Physical "Earth Tan")
world_topo = alt.topo_feature("https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json", "countries")
background = alt.Chart(world_topo).mark_geoshape(
    fill='#e6e1d3',
    stroke='white',
    strokeWidth=0.5
).project(type=proj_type)

# C. Continent Labels (Dark Grey for contrast)
continent_data = pd.DataFrame({
    'Continent': ['North America', 'South America', 'Europe', 'Africa', 'Asia', 'Oceania'],
    'Latitude':  [50,              -15,             54,       5,        40,     -25],
    'Longitude': [-100,            -60,             15,       20,       90,     140]
})

continent_labels = alt.Chart(continent_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=13,
    fontWeight='bold',
    opacity=0.4,
    color='#2c3e50' # Dark Slate Grey
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    text='Continent:N'
)

# D. Bubbles (Updated with "Heat" scale)
bubbles = alt.Chart(df_final).mark_circle(
    stroke='white',
    strokeWidth=0.5,
    opacity=0.85
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',

    # SIZE LEGEND (Fixed Right)
    size=alt.Size(
        'CO2:Q',
        scale=alt.Scale(domain=[0, df_final['CO2'].max()], range=[10, 1500]),
        legend=alt.Legend(
            title='CO2 Emissions (Mt)',
            orient='right',
            symbolFillColor='gray',
            symbolStrokeColor='white',
            symbolStrokeWidth=0.5
        )
    ),

    # COLOR LEGEND (Switched to "Yellow-Orange-Red" for realistic Heat Map feel)
    color=alt.Color(
        'GINI:Q',
        scale=alt.Scale(scheme='yelloworangered', domain=[25, 65]),
        title='Inequality (GINI)',
        legend=alt.Legend(orient='right')
    ),

    tooltip=[
        alt.Tooltip('Country Name', title='Country'),
        alt.Tooltip('year', title='Year'),
        alt.Tooltip('GINI', format='.1f'),
        alt.Tooltip('CO2', format=',.0f', title='CO2 Emissions (Mt)')
    ]
).add_params(
    select_year
).transform_filter(
    select_year
)

# E. Combine Layers
final_globe_map = alt.layer(
    sphere,
    graticule,
    background,
    continent_labels,
    bubbles
).properties(
    width=800,
    height=500,
    title="Global Inequality vs CO2 Emissions (Atlas Style)"
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start'
).configure_legend(
    titleFontSize=12,
    labelFontSize=11,
    padding=10
)

final_globe_map

alt.LayerChart(...)

In [11]:

# ======================================
# 7. CORRELATION ANALYSIS: Scatter Plot with Regression Line
# ======================================

# A. Slider for Time Travel
slider = alt.binding_range(min=min_year, max=max_year, step=1, name='Select Year: ')
select_year = alt.selection_point(fields=['year'], bind=slider, value=2000)

# B. Base Chart
base = alt.Chart(df_final).encode(
    x=alt.X('GINI:Q', title='Inequality (GINI Index)', scale=alt.Scale(domain=[20, 70])),
    y=alt.Y('CO2:Q', title='CO2 Emissions (Log Scale)', scale=alt.Scale(type='log', domain=[1, 15000]))
).add_params(
    select_year
).transform_filter(
    select_year
)

# C. The Scatter Points
points = base.mark_circle(size=100, opacity=0.7, stroke='white', strokeWidth=0.5).encode(
    color=alt.Color('Continent:N', legend=alt.Legend(orient='bottom')),
    tooltip=['Country Name', 'GINI', 'CO2']
)

# D. The Regression Line (The "Correlation")
# This calculates the mathematical trend
line = base.transform_regression(
    'GINI', 'CO2', method='log'
).mark_line(
    color='red',
    size=4,
    strokeDash=[5, 5]
)

# E. Combine
correlation_chart = (points + line).properties(
    width=700,
    height=400,
    title="Is there a Correlation? (Regression Analysis)"
)

correlation_chart

alt.LayerChart(...)

In [12]:
# ... [Keep Sections 1-5 exactly the same] ...

# ======================================
# 8. TRAJECTORY ANALYSIS: The "Development Paths"
# ======================================

# 1. Filter for Key Countries to tell the story
# (Plotting all 200 countries would look like spaghetti)
countries_of_interest = [
    'United States', 'China', 'India',
    'Germany', 'Brazil', 'South Africa',
    'Russian Federation', 'Japan'
]

df_trails = df_final[df_final['Country Name'].isin(countries_of_interest)].copy()

# 2. Base Chart
base = alt.Chart(df_trails).encode(
    x=alt.X('GINI:Q', title='Inequality (GINI Index)', scale=alt.Scale(domain=[20, 70])),
    y=alt.Y('CO2:Q', title='CO2 Emissions (Log Scale)', scale=alt.Scale(type='log', domain=[100, 15000])),
    color=alt.Color('Country Name:N', legend=alt.Legend(title="Country Path")),
    tooltip=['Country Name', 'year', 'GINI', 'CO2']
)

# 3. Layer A: The "Paths" (Lines connecting years)
# We sort by year so the line draws chronologically
trails = base.mark_line(
    opacity=0.6,
    strokeWidth=3
).encode(
    order='year'
)

# 4. Layer B: The "Years" (Dots on the path)
points = base.mark_circle(
    size=50,
    opacity=0.8
)

# 5. Layer C: Labels for the Start (1990) and End (2020)
# This helps us see the direction of movement
text_start = base.transform_filter(
    (alt.datum.year == df_trails['year'].min())
).mark_text(
    align='left', dx=5, dy=-5, text='1990', fontSize=10, fontWeight='bold'
)

text_end = base.transform_filter(
    (alt.datum.year == df_trails['year'].max())
).mark_text(
    align='left', dx=5, dy=-5, text='2020', fontSize=12, fontWeight='bold'
)

# 6. Combine
trajectory_chart = (trails + points + text_start + text_end).properties(
    width=700,
    height=500,
    title="Development Paths: How Countries Evolved (1990-2020)"
).configure_axis(
    grid=True,
    gridOpacity=0.3
)

trajectory_chart

alt.LayerChart(...)